### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Metabolic_Rate/GSE122279'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import json

# 1. Gene Expression Data Availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# Data availability
trait_row = 12  # The trait 'Metabolic_Rate' is stored as 'basal metabolic rate'
age_row = None  # 'age' is not found in the provided data
gender_row = None  # 'gender' is not found in the provided data

# Data type conversion functions
def convert_trait(value):
    try:
        return float(value.split(': ')[1])  # continuous data type
    except:
        return None

def convert_age(value):
    return None  # Not applicable

def convert_gender(value):
    return None  # Not applicable

# 3. Save Metadata
def save_cohort_info(study_id, path, is_gene_available, is_clinical_data_available):
    cohort_info = {
        'study_id': study_id,
        'is_gene_available': is_gene_available,
        'is_clinical_data_available': is_clinical_data_available
    }
    with open(path, 'w') as f:
        json.dump(cohort_info, f)

save_cohort_info('GSE122279', './preprocessed/Metabolic_Rate/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Metabolic_Rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Metabolic_Rate/trait_data/GSE122279.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identifiers and gene symbols determination
identifier_key = 'ID'
gene_symbol_key = 'EntrezGeneSymbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and update gene_data
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Metabolic_Rate/gene_data/GSE122279.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Metabolic_Rate')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
json_path = './preprocessed/Metabolic_Rate/cohort_info.json'
save_cohort_info('GSE122279', json_path, is_gene_available=True, is_trait_available=True, is_biased=trait_biased, df=merged_data, note='')

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Metabolic_Rate/GSE122279.csv'
    unbiased_merged_data.to_csv(csv_path)
